<a href="https://colab.research.google.com/github/Amodtfx/KRLSTM/blob/main/BTC_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install google-colab-shell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for google-colab-shell: filename=google_colab_shell-0.2-py3-none-any.whl size=4123 sha256=951e038791bd79a52054dbb2b816e73b2c4450ee4c7b7b3cdd9dded9981e7cc8
  Stored in directory: /root/.cache/pip/wheels/18/28/39/256dc5595d46f8781c8a52a86e31754b28168b2aa15d5d68fd
Successfully built google-colab-shell


In [2]:
# import the module once
from google_colab_shell import getshell

In [3]:
## Anytime you want to open a terminal

getshell()

getshell(height=400) # custom height of the terminal

In [5]:
# for install package on colab worksheet!!!
#run from kosole
#!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
#!tar -xzf ta-lib-0.4.0-src.tar.gz
#!cd ta-lib/
#!ls
#! ./configure
#! make
#! make install

#### For download data
# cd ..
# mkdir data
# cd data
# end comment

!!pip install ta-lib

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting ta-lib',
 '  Downloading TA-Lib-0.4.25.tar.gz (271 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/272.0 KB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b \x1b266.2/272.0 KB\x1b \x1b9.0 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b272.0/272.0 KB\x1b \x1b7.0 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h  Installing build dependencies ... \x1b[?25l\x1b[?25hdone',
 '  Getting requirements to build wheel ... \x1b[?25l\x1b[?25hdone',
 '  Installing backend dependencies ... \x1b[?25l\x1b[?25hdone',
 '  Preparing metadata (pyproject.toml) ... \x1b[?25l\x1b[?25hdone',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.8/dist-packages (from ta-lib) (1.21.6)',
 'Building wheels for collected packages: ta-lib',
 '  Building wheel fo

In [6]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
import numpy
import talib
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Bidirectional
import os
import warnings

In [22]:
#Download data
!wget https://raw.githubusercontent.com/Amodtfx/KRLSTM/a5d2486806ca1528903efe6510884589e2bf29c5/data/XBTEUR_2018.csv

--2023-01-24 17:41:20--  https://raw.githubusercontent.com/Amodtfx/KRLSTM/a5d2486806ca1528903efe6510884589e2bf29c5/data/XBTEUR_2018.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 86621 (85K) [text/plain]
Saving to: ‘XBTEUR_2018.csv’

XBTEUR_2018.csv     100%[===================>]  84.59K  --.-KB/s    in 0.01s   

2023-01-24 17:41:20 (5.93 MB/s) - ‘XBTEUR_2018.csv’ saved [86621/86621]



In [23]:
!ls


XBTEUR_2018.csv


In [8]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Hide messy TensorFlow warnings
warnings.filterwarnings("ignore") 		#Hide messy Numpy warnings

# transform series into train and test sets for supervised learning
def prepare_data(series, n_lag, n_seq, rsi_14, mfi_14, mom_14, plus_dm_14, dx_14, cci_14, aaron_14, cmo_14, roc_14, rorc_14, will_14, bop, ad, obv, avg_price, med_price, typ_price, wcl_price, atr_14, natr_14, trange, mid_price):    
    # rescale values to 0, 1
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    values_close = series['Close'].values
    #values_close_max = max(values_close)*2.5
    values_close_max = 20000
    values_close = numpy.append([values_close_max,0], values_close)      # add boundaries 
    values_close = values_close.reshape(len(values_close), 1) 
    scaled_values_close = scaler.fit_transform(values_close) 			 # apply scaling 
    scaled_values_close = numpy.delete(scaled_values_close,[0,1]) 	     # remove boundaries

    values_volume = series['Volume'].values
    #values_volume_max = max(values_volume)*2.5
    values_volume_max = 20000
    values_volume = numpy.append([values_volume_max,0], values_volume)  # add boundaries 
    values_volume = values_volume.reshape(len(values_volume), 1) 
    scaled_values_volume = scaler.fit_transform(values_volume) 		    # apply scaling 
    scaled_values_volume = numpy.delete(scaled_values_volume,[0,1]) 	# remove boundaries
    
    values_rsi = rsi_14.reshape(len(scaled_values_close), 1)
    scaled_values_rsi = scaler.fit_transform(values_rsi)

    values_mfi = mfi_14.reshape(len(scaled_values_close), 1)
    scaled_values_mfi = scaler.fit_transform(values_mfi)
    
    values_mom = mom_14.reshape(len(scaled_values_close), 1)
    scaled_values_mom = scaler.fit_transform(values_mom)
    
    values_plus_dm = plus_dm_14.reshape(len(scaled_values_close), 1)
    scaled_values_plus_dm = scaler.fit_transform(values_plus_dm)
    
    values_dx = dx_14.reshape(len(scaled_values_close), 1)
    scaled_values_dx = scaler.fit_transform(values_dx)
    
    values_cci = cci_14.reshape(len(scaled_values_close), 1)
    scaled_values_cci = scaler.fit_transform(values_cci) 

    values_aaron = aaron_14.reshape(len(scaled_values_close), 1)
    scaled_values_aaron = scaler.fit_transform(values_aaron) 
    
    values_cmo = cmo_14.reshape(len(scaled_values_close), 1)
    scaled_values_cmo = scaler.fit_transform(values_cmo)
    
    values_roc = roc_14.reshape(len(scaled_values_close), 1)
    scaled_values_roc = scaler.fit_transform(values_roc)
    
    values_rorc = rorc_14.reshape(len(scaled_values_close), 1)
    scaled_values_rorc = scaler.fit_transform(values_rorc)

    values_will = will_14.reshape(len(scaled_values_close), 1)
    scaled_values_will = scaler.fit_transform(values_will)
    
    values_bop = bop.reshape(len(scaled_values_close), 1)
    scaled_values_bop = scaler.fit_transform(values_bop)
    
    values_ad = ad.reshape(len(scaled_values_close), 1)
    scaled_values_ad = scaler.fit_transform(values_ad)
    
    values_obv = obv.reshape(len(scaled_values_close), 1)
    scaled_values_obv = scaler.fit_transform(values_obv)
    
    values_avg_price = avg_price.reshape(len(scaled_values_close), 1)
    scaled_values_avg_price = scaler.fit_transform(values_avg_price)
    
    values_med_price = med_price.reshape(len(scaled_values_close), 1)
    scaled_values_med_price = scaler.fit_transform(values_med_price)
    
    values_typ_price = typ_price.reshape(len(scaled_values_close), 1)
    scaled_values_typ_price = scaler.fit_transform(values_typ_price)
    
    values_wcl_price = wcl_price.reshape(len(scaled_values_close), 1)
    scaled_values_wcl_price = scaler.fit_transform(values_wcl_price)
    
    values_atr_14 = atr_14.reshape(len(scaled_values_close), 1)
    scaled_values_atr_14 = scaler.fit_transform(values_atr_14)

    values_natr_14 = natr_14.reshape(len(scaled_values_close), 1)
    scaled_values_natr_14 = scaler.fit_transform(values_natr_14)

    values_trange = trange.reshape(len(scaled_values_close), 1)
    scaled_values_trange = scaler.fit_transform(values_trange)
    
    values_mid_price = mid_price.reshape(len(scaled_values_close), 1)
    scaled_values_mid_price = scaler.fit_transform(values_mid_price)
     
    # transform data to be stationary
    #--------------------------------------------
    diff_series_close = difference(scaled_values_close, 1)
    diff_values_close = diff_series_close.values.reshape(len(diff_series_close), 1)
    
    diff_series_volume = difference(scaled_values_volume, 1)
    diff_values_volume = diff_series_volume.values.reshape(len(diff_series_volume), 1)
  
    diff_series_rsi = difference(scaled_values_rsi, 1)
    diff_values_rsi = diff_series_rsi.values.reshape(len(diff_series_rsi), 1)
    
    diff_series_mfi = difference(scaled_values_mfi, 1)
    diff_values_mfi = diff_series_mfi.values.reshape(len(diff_series_mfi), 1)
    
    diff_series_mom = difference(scaled_values_mom, 1)
    diff_values_mom = diff_series_mom.values.reshape(len(diff_series_mom), 1)
    
    diff_series_plus_dm = difference(scaled_values_plus_dm, 1)
    diff_values_plus_dm = diff_series_plus_dm.values.reshape(len(diff_series_plus_dm), 1)
    
    diff_series_dx = difference(scaled_values_dx, 1)
    diff_values_dx = diff_series_dx.values.reshape(len(diff_series_dx), 1)
    
    diff_series_cci = difference(scaled_values_cci, 1)
    diff_values_cci = diff_series_cci.values.reshape(len(diff_series_cci), 1) 
    
    diff_series_aaron = difference(scaled_values_aaron, 1)
    diff_values_aaron = diff_series_aaron.values.reshape(len(diff_series_aaron), 1) 
     
    diff_series_cmo = difference(scaled_values_cmo, 1)
    diff_values_cmo = diff_series_cmo.values.reshape(len(diff_series_cmo), 1)
    
    diff_series_roc = difference(scaled_values_roc, 1)
    diff_values_roc = diff_series_roc.values.reshape(len(diff_series_roc), 1)
    
    diff_series_rorc = difference(scaled_values_rorc, 1)
    diff_values_rorc = diff_series_rorc.values.reshape(len(diff_series_rorc), 1)
    
    diff_series_will = difference(scaled_values_will, 1)
    diff_values_will = diff_series_will.values.reshape(len(diff_series_will), 1)
    
    diff_series_bop = difference(scaled_values_bop, 1)
    diff_values_bop = diff_series_bop.values.reshape(len(diff_series_bop), 1)
    
    diff_series_ad = difference(scaled_values_ad, 1)
    diff_values_ad = diff_series_ad.values.reshape(len(diff_series_ad), 1)
    
    diff_series_obv = difference(scaled_values_obv, 1)
    diff_values_obv = diff_series_obv.values.reshape(len(diff_series_obv), 1)
    
    diff_series_avg_price = difference(scaled_values_avg_price, 1)
    diff_values_avg_price = diff_series_avg_price.values.reshape(len(diff_series_avg_price), 1)
    
    diff_series_med_price = difference(scaled_values_med_price, 1)
    diff_values_med_price = diff_series_med_price.values.reshape(len(diff_series_med_price), 1)
    
    diff_series_typ_price = difference(scaled_values_typ_price, 1)
    diff_values_typ_price = diff_series_typ_price.values.reshape(len(diff_series_typ_price), 1)
    
    diff_series_wcl_price = difference(scaled_values_wcl_price, 1)
    diff_values_wcl_price = diff_series_wcl_price.values.reshape(len(diff_series_wcl_price), 1)
    
    diff_series_trange = difference(scaled_values_trange, 1)
    diff_values_trange = diff_series_trange.values.reshape(len(diff_series_trange), 1)

    diff_series_natr_14 = difference(scaled_values_natr_14, 1)
    diff_values_natr_14 = diff_series_natr_14.values.reshape(len(diff_series_natr_14), 1)

    diff_series_atr_14 = difference(scaled_values_atr_14, 1)
    diff_values_atr_14 = diff_series_atr_14.values.reshape(len(diff_series_atr_14), 1)
    
    diff_series_mid_price = difference(scaled_values_mid_price, 1)
    diff_values_mid_price = diff_series_mid_price.values.reshape(len(diff_series_mid_price), 1)
    #--------------------------------------------
    together = numpy.concatenate((diff_values_mid_price, diff_values_atr_14, diff_values_natr_14, diff_values_trange, diff_values_wcl_price, diff_values_typ_price, diff_values_med_price, diff_values_avg_price, diff_values_obv, diff_values_ad, diff_values_bop, diff_values_will, diff_values_rorc, diff_values_roc, diff_values_cmo, diff_values_aaron, diff_values_cci, diff_values_dx, diff_values_plus_dm, diff_values_mom, diff_values_mfi, diff_values_volume, diff_values_rsi , diff_values_close), axis=1)
    #--------------------------------------------
    # transform into supervised learning problem X, y
    supervised = series_to_supervised(together, n_lag, n_seq)
    train = supervised.values

    return scaler, train

In [9]:
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return Series(diff)


In [10]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i).iloc[:,-1])
        if i == 0:
            names += ['var(t)']
        else:
            names += ['var(t+%d)' % i]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg



In [26]:
def fit_lstm(train, n_lag, n_batch, nb_epoch, n_neurons, n_neurons2, n_neurons3):
    # reshape training into [samples, timesteps, features]
    X, y = train[:, :-n_seq], train[:, -n_seq:]
    X = X.reshape(X.shape[0], n_lag, n_features)
    model = Sequential()
    #First recurrent layer with dropout
    model.add(Bidirectional(LSTM(n_neurons, return_sequences=True), batch_input_shape=(n_batch, X.shape[1], X.shape[2])))
    model.add(Dropout(0.3))
    #Second recurrent layer with dropout
    model.add(Bidirectional(LSTM((n_neurons2), return_sequences=True)))
    model.add(Dropout(0.3))
    #Third recurrent layer
    model.add(Bidirectional(LSTM(n_neurons3, return_sequences=False)))
    #Output layer (returns the predicted value)
    model.add(Dense(y.shape[1],activation='linear'))
    #Set loss function and optimizer
    model.compile(loss='mse', optimizer='adam')
    # fit network
    model.fit(X, y, epochs=nb_epoch, batch_size=n_batch, verbose=1, shuffle=False)
    return model


In [24]:
# load dataset
load_data = read_csv('XBTEUR_2018.csv', header=0)

In [30]:
# configure
n_lag = 5		 # okno (z kolika předchozích hodnot)
n_seq = 3		 # počet predikovaných kroků t+n
n_features = 24  # Close, Volume, RSI..
n_epochs = 2000
n_batch = 1
n_neurons = 32
n_neurons2 = 64
n_neurons3 = 32
rsi_n_day = 14

series_data = load_data[rsi_n_day:].copy()
series = series_data.reset_index(drop=True)

rsi_14 = talib.RSI(numpy.array(load_data['Close']), rsi_n_day)  	# Vypocet RSI //+14 hodnot
rsi_14 = rsi_14[rsi_n_day:]										    # Nan value trim

mfi_14 = talib.MFI(numpy.array(load_data['Max']),numpy.array(load_data['Min']),numpy.array(load_data['Close']),numpy.array(load_data['Volume']), rsi_n_day)
mfi_14 = mfi_14[rsi_n_day:]	

mom_14 = talib.MOM(numpy.array(load_data['Close']), rsi_n_day)
mom_14 = mom_14[rsi_n_day:]

plus_dm_14 = talib.PLUS_DM(numpy.array(load_data['Max']), numpy.array(load_data['Min']), rsi_n_day)	
plus_dm_14 = plus_dm_14[rsi_n_day:]

dx_14 = talib.DX(numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']), rsi_n_day)
dx_14 = dx_14[rsi_n_day:]

cci_14 = talib.CCI(numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']), rsi_n_day)
cci_14 = cci_14[rsi_n_day:]

aaron_14 = talib.AROONOSC(numpy.array(load_data['Max']), numpy.array(load_data['Min']), rsi_n_day)
aaron_14 = aaron_14[rsi_n_day:]

cmo_14 = talib.CMO(numpy.array(load_data['Close']), rsi_n_day)
cmo_14 = cmo_14[rsi_n_day:]

roc_14 = talib.ROC(numpy.array(load_data['Close']), rsi_n_day)
roc_14 = roc_14[rsi_n_day:]

rorc_14 = talib.ROCR(numpy.array(load_data['Close']), rsi_n_day)
rorc_14 = rorc_14[rsi_n_day:]

will_14 = talib.WILLR(numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']), rsi_n_day)
will_14 = will_14[rsi_n_day:]

bop = talib.BOP(numpy.array(load_data['Open']), numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']))
bop = bop[rsi_n_day:]

ad = talib.AD(numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']), numpy.array(load_data['Volume']))
ad = ad[rsi_n_day:]

obv = talib.OBV(numpy.array(load_data['Close']), numpy.array(load_data['Volume']))
obv = obv[rsi_n_day:]

avg_price = talib.AVGPRICE(numpy.array(load_data['Open']), numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']))
avg_price = avg_price[rsi_n_day:]

med_price = talib.MEDPRICE(numpy.array(load_data['Max']), numpy.array(load_data['Min']))
med_price = med_price[rsi_n_day:]

typ_price = talib.TYPPRICE(numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']))
typ_price = typ_price[rsi_n_day:]

wcl_price = talib.WCLPRICE(numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']))
wcl_price = wcl_price[rsi_n_day:]

atr_14 = talib.ATR(numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']), rsi_n_day)
atr_14 = atr_14[rsi_n_day:]

natr_14 = talib.NATR(numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']), rsi_n_day)
natr_14 = natr_14[rsi_n_day:]

trange = talib.TRANGE(numpy.array(load_data['Max']), numpy.array(load_data['Min']), numpy.array(load_data['Close']))
trange = trange[rsi_n_day:]

mid_price = talib.MIDPRICE(numpy.array(load_data['Max']), numpy.array(load_data['Min']), rsi_n_day)
mid_price = mid_price[rsi_n_day:]


In [31]:
scaler, train = prepare_data(series, n_lag, n_seq, rsi_14, mfi_14, mom_14, plus_dm_14, dx_14, cci_14, aaron_14, cmo_14, roc_14, rorc_14, will_14, bop, ad, obv, avg_price, med_price, typ_price, wcl_price, atr_14, natr_14, trange, mid_price)


In [32]:
model = fit_lstm(train, n_lag, n_batch, n_epochs, n_neurons, n_neurons2, n_neurons3)



ValueError: ignored

In [ ]:
model.save("BTC_basic.h5", overwrite=True)